**Notebook content:**
- split events by thier category (which was given mostly manually) 
- 

In [104]:
%reset -f 

In [3]:
import sys
import pandas as pd
import numpy as np
import datetime
from IPython.display import display

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 200)
#pd.set_option('display.float_format', lambda x: '%.f' % x)
pd.options.display.precision = 4

#import sys
np.set_printoptions(threshold=sys.maxsize) #- print the full NumPy array

from myDefs.defs import *

# visualization
import seaborn as sns
import matplotlib.pyplot as plt 
from matplotlib.pyplot import figure
%matplotlib inline


In [4]:
out_file = DATA_PATH + "organizeEvents/category_{}.xlsx"


In [40]:
file = DATA_PATH + "organizeEvents/"+ "all_events_categories.xlsx"
df = pd.read_excel(file, index=False)
df.head()

,eventName,count,category,type,feature_name,description,unitOfMeasure,dValues_description,iValues_description,sValue_values,eventDesc
0,האם יש לחולה טובוס,947,general,bool,NaN,NaN,NaN,NaN,NaN,"['0', '1']",NaN
1,החולה מונשם,295,general,bool,mechanical ventilation,NaN,NaN,NaN,NaN,"['0', '1']",NaN
2,"רופא/ה מבצע/ת הפעולה לבוש/ה ב: כובע,מסיכה,פה ו...",240,general,bool,NaN,NaN,NaN,NaN,NaN,['1'],NaN
3,אחות/רופא/ה מסייע/ת עוטה מסיכה פה ואף?,78,general,bool,mask,NaN,NaN,NaN,NaN,"['0', '1']",NaN
4,כל מבצעי הפעולה חיטאו ידים לפני עטית כפפות?,78,general,bool,hand wash,NaN,NaN,NaN,NaN,"['0', '1']",NaN


In [41]:
df.shape

(3755, 11)

In [42]:
df['category'].value_counts()

general         2266
drug            1287
laboratory        94
physical          58
nutrition         27
microbiology      12
drain              5
procedure          3
surgeon            1
infusion           1
imaging            1
Name: category, dtype: int64

In [77]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
def f(x):
    s = x.strip("[")
    s = s.strip("]")
    s = s.split(',')
    if s[0] == 'nan':
        return False
    print(s[0])
    return is_number(s[0])



inx = df.loc[df['sValue_values'].notna(), 'sValue_values'].apply(f)
inx = df['sValue_values'].notna() & inx
df.loc[inx, :]# = 'numeric-text'

'0'
'0'
'1'
'0'
'0'
'0'
'0'
'+++++ '
'+++++ '
'<1.5 '
'<3.3 '
"'++"
"'Normal"
'>1500 '
'TT'
'לא יציב'
"'15"
"'++"
'בעית רקע'
'בעית רקע'
'עצמאי'
'1'
'בעית רקע'
'ארטא אורטופדית א'
'היפוקסורקימ תואצותל ןיתמהל שי'
'פעיל'
'בעית רקע'
'מעורפל'
"'מעוכל"
'1'
'סביר'
'0'
"'21910"
'0'
'סיכום ביניים'
'PCMV'
"'6"
'<150 '
"'00522228614"
'1'
'בעית רקע'
"'לא"
'<18.5'
'בעית רקע'
'ספונטאני'
'יציב '
"'alinor"
'בעית רקע'
"'מוריה"
'PICC'
"'ערבית"
'בעית רקע'
"'אורי"
'דרך הפה'
'PCV/PS'
"'75"
'0'
"'פרופול+פנטניל"
'בעית רקע'
'אי שקט'
"'15"
'בעית רקע'
'ENTEROCOCCUS FAECALIS'
"'138"
"'55397"
"'0.09"
"'201811612"
"'11"
"'165"
"'120"
'אין'
"'38973"
"'יוני"
"'תקינות"
'אין'
'בעית רקע'
'ורוד מדמם'
"'אנאורי"
'בעית רקע'
"'35.5"
'פתוח'
"'106"
'++ '
'1'
"'7"
'בעית רקע'
'בעית רקע'
"'ניתוח"
"'17"
"'9.3"
'מופחת'
'חיובי5.0'
"'מעורפל"
"'נוראדרנלין."
"'בבטן"
'בעית רקע'
"'ZUNAT"
"'S1"
"'אורולוגיה"
'בעית רקע'
'פעיל'
"'ללא."
"'30"
'פעיל'
'פעיל'
'בעית רקע'
'בעית רקע'
"'18"
'פעיל'
"'מודאגים"
'שלילי'
'Cytoplasmic (1:80) '
'שלילי'
'C-

,eventName,count,category,type,feature_name,description,unitOfMeasure,dValues_description,iValues_description,sValue_values,eventDesc


In [37]:
#types

inx = (df['dValues_description'].notna() | df['iValues_description'].notna()) \
    & (df['sValue_values'].isna() & df['eventDesc'].isna())
df.loc[inx, 'type'] = 'numeric'

inx = (df['sValue_values'].notna() | df['eventDesc'].notna()) \
    & (df['dValues_description'].isna() & df['iValues_description'].isna())
df.loc[inx, 'type'] = 'text'


inx = (df['sValue_values'].notna() | df['eventDesc'].notna()) \
    & (df['dValues_description'].notna() | df['iValues_description'].notna())
df.loc[inx, 'type'] = 'mixed'

inx = df['sValue_values'].isin( ["['0', '1']", "['1']", "['1', '0']"]  )
df.loc[inx, 'type'] = 'bool'

In [38]:
inx = df['dValues_description'].str.contains("'std': 0.0,", regex=True, na=False) 
print(inx.sum())
df.loc[inx, 'type'] = 'constant'    

1180


In [34]:
df[df['type'] == 'constant'   ]

,eventName,count,category,type,feature_name,description,unitOfMeasure,dValues_description,iValues_description,sValue_values,eventDesc
12,Noradrenaline (Norepinephrine),2613,drug,constant,NaN,NaN,NaN,"{'count': 2613.0, 'mean': 0.001446927178436802...",NaN,"[nan, 'IV']",NaN
13,FLAGYL vial vial 500 mg,1916,drug,constant,FLAGYL,NaN,['mg'],"{'count': 1916.0, 'mean': 500.0, 'std': 0.0, '...",NaN,NaN,NaN
14,CARDILOC tab tab 2.5 mg,1228,drug,constant,NaN,NaN,['mg'],"{'count': 1228.0, 'mean': 2.5, 'std': 0.0, 'mi...",NaN,NaN,NaN
15,Furosemide (Fusid),1226,drug,constant,NaN,NaN,NaN,"{'count': 1226.0, 'mean': 0.06296402561113848,...",NaN,"[nan, 'IV']",NaN
16,Fentanyl,1081,drug,constant,Fentanyl,NaN,NaN,"{'count': 1081.0, 'mean': 0.001267522967875634...",NaN,"[nan, 'IV']",NaN
...,...,...,...,...,...,...,...,...,...,...,...
3405,SODIUM CHLORIDE 0.9% infus Inject 500 cc(ml)+P...,2,general,constant,NaN,NaN,['cc(ml)'],"{'count': 2.0, 'mean': 500.0, 'std': 0.0, 'min...",NaN,NaN,NaN
3407,BONDORMIN tab 0.250mg 0.25 mg (0.25mg/24 hours),2,general,constant,NaN,NaN,['mg'],"{'count': 2.0, 'mean': 0.25, 'std': 0.0, 'min'...",NaN,NaN,NaN
3408,SODIUM CHLORIDE 0.9% infus Inject 3000 cc(ml) ...,2,general,constant,NaN,NaN,['cc(ml)'],"{'count': 2.0, 'mean': 3000.0, 'std': 0.0, 'mi...",NaN,NaN,NaN
3409,SODIUM CHLORIDE 0.9% inj 1000 cc(ml) (1.33cc(...,2,general,constant,NaN,NaN,['cc(ml)'],"{'count': 2.0, 'mean': 1000.0, 'std': 0.0, 'mi...",NaN,NaN,NaN


In [21]:
inx = df['category'].isna()
df.loc[inx, 'category'] = 'general'

In [72]:
file = DATA_PATH + "organizeEvents/"+ "all_events_categories.xlsx"
df.to_excel(file, index=False)

In [192]:
inx = df['dValues_description'].isna() & \
       df['iValues_description'].isna()  ) #(df['sValue_values'].notna() |  df['eventDesc'].notna()

#df.loc[inx & (df['type'] != 'bool'), 'type'] = 'text'
#df.loc[inx & (df['type'] != 'bool')  & (df['category'].isna() | (df['category']=='text')) , 'type']= 'text'
#sub = df.loc[inx,:]

#file = out_file.format('eventDesc')
#sub.to_excel(file, index=False)

In [201]:
df.loc[df['category'] == 'drain', 'type'] = 'numeric'

## Nnutrition category 

In [149]:
#df.loc[df['eventName'].str.contains('ENSURE', regex=True) , 'category'] = 'nutrition'

## Drug category 

In [109]:
#events end with gram / gr / cc(ml), etc, are drugs ----> One-time run
#df.loc[df['eventName'].str.contains(' gram$| gr$| mg$| cc\(ml\)$| mcg$', regex=True) , 'category'] = 'drug'

## empty category

Events which don't have values in onle of columns iValue, dValue, sValue, eventDesc

In [157]:
inx = df['dValues_description'].isna() & \
       df['iValues_description'].isna() & \
       df['sValue_values'].isna() & \
        df['eventDesc'].isna()  

# split to files by category 

In [159]:
categories = ['drug', 'empty', 'laboratory', 'nutrition']
categories = ['laboratory']
for cat in categories:
    sub = df[df['category'] == cat]
    file = out_file.format(cat)
    sub.to_excel(file, index=False)